In [53]:
import pandas as pd
import numpy as np
import re

In [54]:
data = pd.read_csv('../data/attacks.csv', encoding = 'utf-8', encoding_errors = 'ignore')

# Printeamos el DataFrame para visualizar los datos

In [55]:
pd.set_option('display.max_columns', None)

data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [56]:
data.shape

(25723, 24)

# Nos deshacemos de las filas duplicadas

In [57]:
data.drop_duplicates().shape == data.shape

False

In [58]:
data.drop_duplicates(inplace=True)

In [59]:
data.shape

(6312, 24)

# Eliminamos las filas que tengan un porcentaje mayor al 75% de nulos

In [60]:
porcent_null = (data.isnull().sum(axis = 1) / data.shape[1]) * 100 
porcent_bool = porcent_null > 75        
data = data.loc[~porcent_bool, :]       

In [61]:
data.shape

(6302, 24)

# Y reseteamos los indices de nuestro DataFrame

In [62]:
data = data.reset_index(drop = True)
data.tail()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8N, 79W",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN
6301,ND.0001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001,2.0,NaN,NaN


# Ahora empezamos con la limpieza por columnas 

### Columna Type

In [63]:
data['Type'].value_counts()

Type
Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: count, dtype: int64

In [64]:
def limpiar_type(x):

    if pd.isnull(x):
        return x
    elif 'Unprovoked' in x:
        return 'Unprovoked'
    elif 'Provoked' in x:
        return 'Provoked'
    else:
        return 'Unprovoked'

In [65]:
data.Type = data.Type.apply(limpiar_type)
data['Type'].value_counts()

Type
Unprovoked    5724
Provoked       574
Name: count, dtype: int64

### Columna Sex

In [66]:
data['Sex '].value_counts()

Sex 
M      5094
F       637
M         2
N         2
lli       1
.         1
Name: count, dtype: int64

In [67]:
data.rename(columns = {'Sex ': 'Sex'}, inplace=True)

In [68]:
def limpiar_sex(x):

    if pd.isnull(x):
        return x
    elif 'M' in x:
        return 'Male'
    elif 'F' in x:
        return 'Female'
    else:
        return 'Unknown'

In [69]:
data.Sex = data.Sex.apply(limpiar_sex)
data['Sex'].value_counts()

Sex
Male       5096
Female      637
Unknown       4
Name: count, dtype: int64

### Columna Fatal

In [70]:
data['Fatal (Y/N)'].value_counts()

Fatal (Y/N)
N          4293
Y          1388
UNKNOWN      71
 N            7
M             1
2017          1
N             1
y             1
Name: count, dtype: int64

In [71]:
data.rename(columns = {'Fatal (Y/N)': 'Fatal'}, inplace=True)

In [72]:
def limpiar_fatal(x):
    x = str(x).lower().strip()
    if x == 'y':
        return 'Yes'
    elif x == 'n':
        return 'No'
    else:
        return 'Unknown'

In [73]:
data['Fatal'] = data['Fatal'].apply(limpiar_fatal)

In [74]:
data['Fatal'].value_counts()

Fatal
No         4301
Yes        1389
Unknown     612
Name: count, dtype: int64

### Columna Year

In [75]:
data.Year.value_counts().head()

Year
2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
Name: count, dtype: int64

In [76]:
def limpiar_year(fecha):
    if pd.isna(fecha):
        return 0
    elif len(str(int(fecha))) == 4:
        return fecha
    else:
        return 0

In [77]:
data['Year'] = data['Year'].apply(limpiar_year)

In [78]:
data['Year'] = data['Year'].astype(int)
data['Year'] = data['Year'].replace(0, 'Unknown')

In [79]:
data.Year.value_counts().head()

Year
2015       143
2017       136
Unknown    130
2016       130
2011       128
Name: count, dtype: int64

### Columna Country

In [80]:
data['Country'].value_counts().head()

Country
USA                 2229
AUSTRALIA           1338
SOUTH AFRICA         579
PAPUA NEW GUINEA     134
NEW ZEALAND          128
Name: count, dtype: int64

In [81]:
def limpiar_country(pais):
    if re.search(r'\b(OCEAN|SEA|PACIFIC|AFRICA|ASIA|DIEGO|BETWEEN)\b', pais, re.IGNORECASE):
        return 'UNKNOWN'
    else:
        return pais

In [82]:
data['Country'] = data['Country'].str.upper().str.strip().fillna('UNKNOWN')
data['Country'] = data['Country'].str.replace(r'?', '').str.strip()
data['Country'] = data['Country'].str.replace(r'&', '/')
data['Country'] = data['Country'].str.replace('ISLANDAS', 'ISLANDS')
data['Country'] = data['Country'].str.replace('(UAE)', '').str.strip()
data['Country'] = data['Country'].str.replace('(SRI LANKA)', '').str.strip()
data['Country'] = data['Country'].str.replace('DIEGO GARCIA', 'UNKNOWN')
data['Country'] = data['Country'].str.replace('BETWEEN PORTUGAL / INDIA', 'UNKNOWN')

In [83]:
data['Country'] = data['Country'].apply(limpiar_country)

In [84]:
data.Year.value_counts().head()

Year
2015       143
2017       136
Unknown    130
2016       130
2011       128
Name: count, dtype: int64

### Columna Age

In [85]:
data.Age.value_counts().head()

Age
17    154
18    150
19    142
20    141
15    139
Name: count, dtype: int64

In [86]:
def limpiar_age(edad):
       
    if pd.isna(edad):
        return 'Unknown'

    if 'or' in edad:
        edades = re.findall(r'\d+', edad)
        media = (int(edades[0]) + int(edades[1])) / 2
        return int(media)
    
    if 'to' in edad:
        edades = re.findall(r'\d+', edad)
        media = (int(edades[0]) + int(edades[1])) / 2
        return int(media)
   
    if 'month' in edad:
        meses = re.findall(r'\d+', edad)
        return int(int(meses[0]) / 12)
    
    if not re.match(r'^\d+|\d+\.\d+$', edad):
        return 'Unknown'

    edad = re.sub(r'[\'"\?]', '', edad)
    edad = re.sub(r'[a-zA-Z]', '', edad)

    return edad

In [87]:
data['Age'] = data['Age'].apply(limpiar_age)
data['Age'] = data['Age'].astype(str)

In [88]:
data['Age'] = data['Age'].str.replace('21 & ', '21 & Unknown')
data['Age'] = data['Age'].str.replace('7      &    31', '7 & 31')
data['Age'] = data['Age'].str.replace('37, 67, 35, 27,   & 27', '37, 67, 35, 27, Unknown & 27')
data['Age'] = data['Age'].str.replace('21, 34,24 & 35', '21, 34, 24 & 35')

In [89]:
data.Age.value_counts().head()

Age
Unknown    2871
17          155
20          150
18          150
19          143
Name: count, dtype: int64

### Columna Time

In [90]:
data['Time'].value_counts().head()

Time
Afternoon    187
11h00        128
Morning      121
12h00        109
15h00        108
Name: count, dtype: int64

In [91]:
def limpiar_time(hora):
       
    if pd.isna(hora):
        return 'Unknown'
    
    hora = hora.strip()
    if not re.match(r'^\d+|\d+\.\d+$', hora):
        return 'Unknown'

    if len(hora) == 3:
        hora = f"{hora[:1]}h{hora[1:]:02}"
    elif len(hora) == 4:
        hora = f"{hora[:2]}h{hora[2:]:02}"
    else:
        hora = 'Unknown'
        
    hora = hora.replace('--', '/')
    hora = hora.replace('  ', ' / ')
    hora = re.sub(r'(-|\/).*', '', hora)
    hora = re.sub(r'\D', '', hora)

    return hora

In [92]:
data["Time"] = data["Time"].apply(limpiar_time)

In [93]:
data['Time'].value_counts().head()

Time
Unknown    3943
           2352
1600          2
900           1
1300          1
Name: count, dtype: int64

### Columna Area

In [94]:
data['Area'].value_counts().head()

Area
Florida            1037
New South Wales     486
Queensland          311
Hawaii              298
California          290
Name: count, dtype: int64

In [95]:
def limpiar_area(zona):

    if pd.isna(zona):
        return 'Unknown'
    
    if not zona:
        return 'Unknown'

    zona = re.sub(r'[\'"\,?:]|to|miles', '', zona)
    zona = re.sub(r'\b[a-z]\b', '', zona)
    zona = re.sub(r'\d', '', zona)
    zona = zona.title()
    zona = zona.strip()

    return zona

In [96]:
data["Area"] = data["Area"].apply(limpiar_area)

In [97]:
data['Area'] = data['Area'].str.replace('N W', 'Unknown')
data['Area'] = data['Area'].str.replace('S / E', 'Unknown')
data['Area'] = data['Area'].str.replace('.N-.W', 'Unknown')
data['Area'] = data['Area'].str.replace('N E', 'Unknown')

In [98]:
data['Area'].value_counts().head()

Area
Florida            1038
New South Wales     486
Unknown             462
Queensland          312
Hawaii              298
Name: count, dtype: int64